# CSE 258 - HW 1
Jin Dai / A92408103

## Classification (week 2)

In [21]:
from urllib.request import urlopen

In [22]:
def parse_data_from_url(fname):
  for l in urlopen(fname):
    yield eval(l)

In [23]:
beer_reviews = list(parse_data_from_url("https://cseweb.ucsd.edu/classes/fa21/cse258-b/data/beer_50000.json"))

In [24]:
beer_reviews[0]

{'review/appearance': 2.5,
 'beer/style': 'Hefeweizen',
 'review/palate': 1.5,
 'review/taste': 1.5,
 'beer/name': 'Sausa Weizen',
 'review/timeUnix': 1234817823,
 'beer/ABV': 5.0,
 'beer/beerId': '47986',
 'beer/brewerId': '10325',
 'review/timeStruct': {'isdst': 0,
  'mday': 16,
  'hour': 20,
  'min': 57,
  'sec': 3,
  'mon': 2,
  'year': 2009,
  'yday': 47,
  'wday': 0},
 'review/overall': 1.5,
 'review/text': 'A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.',
 'user/profileName': 'stcules',
 'review/aroma': 2.0}

### Question. 7

In [25]:
data = [d for d in beer_reviews if 'review/overall' in d and 'review/text' in d]

In [26]:
def featurize(data, featurizer):
    return [featurizer(d) for d in data]

In [27]:
X = featurize(data, lambda d : [1, len(d['review/text'])])

In [28]:
X[0:10]

[[1, 262],
 [1, 338],
 [1, 396],
 [1, 401],
 [1, 1145],
 [1, 728],
 [1, 471],
 [1, 853],
 [1, 472],
 [1, 1035]]

In [29]:
def extract_labels(data):
    return [d['review/overall'] for d in data if d['review/overall'] >= 4]

In [30]:
y = extract_labels(data)

In [31]:
y[0:10]

[4.0, 4.0, 4.5, 5.0, 4.0, 4.0, 4.0, 4.5, 4.0, 4.0]

In [ ]:
from sklearn import linear_model

In [ ]:
mod = linear_model.LogisticRegression(C=1.0, class_weight='balanced')
mod.fit(X,y)